In [ ]:
import requests
import sqlite3

In [ ]:
client_id = '626'
client_secret = '3fa520ce21bab7c1d81a15c9d0676e36'

In [ ]:
credentials_endpt = 'https://booking.library.gwu.edu/1.1/oauth/token'
calendars_endpt = 'https://booking.library.gwu.edu/1.1/calendars'
bookings_endpt = 'https://booking.library.gwu.edu/1.1/space/bookings'

In [ ]:
def create_cred_body(client_id: str, client_secret: str):
    '''Creates the body of a POST request for obtaining an access token, given a client secret and credentials.'''
    return {'client_id': client_id,
           'client_secret': client_secret,
           'grant_type': 'client_credentials'}

In [ ]:
def get_access_token(cred_body: dict):
    '''Retrieves an access token from the LibCal API, given a request body as specified.'''
    try:
        resp = requests.post(credentials_endpt, json=cred_body)
        resp.raise_for_status()
        token_data = resp.json()
        if 'error' in token_data:
            raise Exception(token_data)
            #return resp
        return token_data
    except Exception as e:
        print(e)
        raise

In [ ]:
cred_body = create_cred_body(client_id, client_secret)
token_data = get_access_token(cred_body)

In [ ]:
token_data

In [ ]:
def create_header(token_data: dict):
    '''Creates an authorization header with the supplied token from the credentials API.'''
    try:
        token = token_data['access_token']
        return {'Authorization': f'Bearer {token}'}
    except KeyError:
        print('Access token missing from token data.')
        raise

In [ ]:
def get_calendars(header: dict):
    '''Retrieves the calendars available in the system, for querying events from a specific calendar.
    Header argument should contain the authorization token.'''
    try:
        resp = requests.get(calendars_endpt, headers=header)
        resp.raise_for_status()
        cal_data = resp.json()
        if 'error' in cal_data:
            raise Exception(cal_data)
        return cal_data
    except Exception as e:
        print('Error getting calendars.')
        raise
    

In [ ]:
header = create_header(token_data)
calendars = get_calendars(header)

In [ ]:
def create_booking_params(date: str = None):
    '''Sets querystr parameters for the space/bookings endpoint. 
    Date should be in the format YYYY-MM-DD.'''
    params = {'formAnswers': 1,
             'limit': 100} # Max value
    if date:
        params['date'] = date
    return params
        

In [ ]:
def get_bookings(header: dict, booking_id: str = None, date: str = None):
    '''Retrieves bookings from LibCal.'''
    if booking_id:
        url = f'{bookings_endpt}/{booking_id}'
    else:
        url = bookings_endpt
    try:
        params = create_booking_params(date)
        resp = requests.get(url, headers=header, params=params)
        resp.raise_for_status()
        bookings_data = resp.json()
        if 'error' in bookings_data:
            raise Exception(bookings_data)
        return bookings_data
    except Exception as e:
        print('Error getting bookings')
        raise

I think the `space/bookings` endpoint is the one to use for the room reservation system. Note that it seems possible to retrieve appointments only for one day at a time. By default, the date is today's date. Another date can be supplied as a querystring parameter.

In [ ]:
get_bookings(header)

In [ ]:
get_bookings(header, date='2020-08-11')